In [1]:
import unittest
import numpy as np
import pandas as pd

Below, we read the metadata about mice fecal samples collected during experimental implementation of the study:

In [2]:
# reading metadata (n=208)
mapping = pd.read_table("./data/haddad_6week_metadata.txt", sep = '\t', dtype = str, index_col = "#SampleID")
mapping.shape

(208, 69)

We load the microbiome feature table:
Microbiome (16S amplicon sequencing) feature table is represented as relative abundance of unique 16S sequences per sample. (generated using default parameters of deblur workflow in QIITA)

In [3]:
microb = pd.read_table("./data/haddad_6week_deblur_otus_unrare.txt", sep = '\t', dtype = str)

In [4]:
#converting microbiome feature table to biom HDF5 format
! biom convert -i ./data/haddad_6week_deblur_otus_unrare.txt \
  -o ./data/haddad_6week_deblur_otus_unrare_hdf5.biom --table-type "OTU table" \
  --to-hdf5

In [25]:
# converting microbiome biom table to QIIME2 artifact (https://docs.qiime2.org/2017.12/interfaces/artifact-api/)
! qiime tools import \
  --input-path ./data/haddad_6week_deblur_otus_unrare_hdf5.biom \
  --type 'FeatureTable[Frequency]' \
  --source-format BIOMV210Format \
  --output-path ./data/haddad_6week_deblur_otus_unrare_hdf5.qza

Next, we rarefy the deblur sotu table to an even sampling depth so that technical variation between sequencing efforts does not mask biological signal. 
- Sampling depth was chosen to be 2000 after interactive inspection of deblur sotu table in QIIME2 viewer. 182 samples (#sequences > 2000) were retained
- We save a biom 2.0 version of rarefied sotu table for downstream analysis

In [26]:
! qiime feature-table rarefy --i-table ./data/haddad_6week_deblur_otus_unrare_hdf5.qza \
--p-sampling-depth 2000 \
--o-rarefied-table ./data/haddad_6week_deblur_otus_rare2k.qza

! qiime tools export \
  ./data/haddad_6week_deblur_otus_rare2k.qza \
  --output-dir ./data/biom_rare2k_exp

Saved FeatureTable[Frequency] to: ./data/haddad_6week_deblur_otus_rare2k.qza


In [27]:
# convert biom table to to tab-delimited
! biom convert -i ./data/biom_rare2k_exp/feature-table.biom \
  -o ./data/haddad_6week_deblur_otus_rare2k.txt --table-type "OTU table" \
  --to-tsv

# update 'microb' dataframe to rarefied feature table
microb = pd.read_table("./data/haddad_6week_deblur_otus_rare2k.txt", sep = '\t', dtype = str, skiprows = 1,
                      index_col = "#OTU ID")

In [ ]:
microb.head(3)

We load the metabolome feature table: Metabolome (untargeted LC-MS/MS) feature table is represented as relative abundance of small molecular features([massToCharge]_[retentionTime]) per sample. (generated using MZmine2)

In [29]:
# read metabolomic feature table
metab = pd.read_table("./data/haddad_6week_allFeatures_nonorm.txt", sep = '\t', dtype = str, index_col = "#featureID")
metab.head(3)

,10422.17.F.6,10422.17.F.2,10422.17.F.5,10422.17.F.7,10422.17.F.10,10422.17.F.4,10422.17.F.3,10422.17.F.8,10422.18.F.6,10422.18.F.2,...,10422.32.F.3,10422.32.F.7,10422.31.F.10,10422.32.F.12,10422.32.F.4,10422.32.F.13,10422.31.F.13,10422.32.F.9,10422.31.F.12,10422.32.F.11
#featureID,,,,,,,,,,,,,,,,,,,,,
132.1020923761221_0.4491537660256411,2.894960860988594E9,9.610168467776846E9,1.182263014985316E10,8.237728873128403E9,8.141911119798736E9,4.635300446851906E9,7.689054931586625E9,7.823334115189182E9,1.6703016660984683E9,1.2743426190491224E10,...,1.222905233156484E10,8.572231265031997E9,9.341332384041187E9,4.2168697214174366E9,5.43359559299014E9,1.3011845333573378E10,1.2588946976281939E10,1.4050915077632828E10,8.432949753442624E9,1.4761605463249516E10
166.0862755041856_0.5858793269230769,1.1305934772556405E9,5.955111906130312E9,6.746907823730342E9,4.111389704534751E9,4.1108703946595316E9,2.5244175646732197E9,4.1095420137630014E9,5.900437305735503E9,4.878848187955626E8,8.976942725740124E9,...,8.423897075956502E9,5.415180143984718E9,5.469345002638374E9,2.1886257254779987E9,3.1254080826838765E9,9.332327131924E9,8.416375498222314E9,1.0852542398524624E10,5.641343370020001E9,1.0800003954566751E10
357.2784138555112_5.010000161030595,1.8926338380370038E9,2.092870323073E9,2.523493777847994E9,1.9688080860177503E9,2.0885578201722806E9,3.3820405383121357E9,2.6865490029608774E9,2.5234752114992456E9,1.0699061940879337E9,1.878606502437749E9,...,1.5165929009018714E9,2.099398512921627E9,2.75377116859438E9,2.282654193257812E9,2.3633356128312488E9,2.650666432169661E9,2.243799569820179E9,2.755781674588057E9,3.5528350605919967E9,2.777342410653752E9


Metabolomics feature table is PQN normalized to internal standard using the code below
(internal standard used: m/z = 278.1896579171936; retention time = 3.8181483173076907 mins)

In [30]:
# PQN normalization
int_stand='278.1896579171936_3.8181483173076907'
ref=pd.to_numeric(metab.loc[int_stand])
se_zero=[]

for feature in metab.index:
    metab.loc[feature]=pd.to_numeric(metab.loc[feature])
    size_effect=np.median(metab.loc[feature]/ref)
    if size_effect==0:
        se_zero.append(feature)
        continue
    else:
        metab.loc[feature]=metab.loc[feature]/size_effect

In [31]:
# save normalized table
metab.to_csv("./data/haddad_6week_allFeatures_pqn.txt", sep='\t')

It is expected that in high-throughput multi-omics experiments, some samples may not generate reliable data needed for further analysis. These samples are dropped during intial quality check. Therefore, we match samples present in metadata, microbiome and metabolome feature tables so as to keep only common samples for further analysis.

In [32]:
def match_samples(mic, met, md):
    mic_samp=set(mic.index)
    met_samp=set(met.index)
    md_samp=set(md.index)
    matched=sorted(list(mic_samp & met_samp & md_samp))
    mic=mic.loc[matched]
    met=met.loc[matched]
    md=md.loc[matched]
    return(mic, met, md)
    

    m1 = pd.DataFrame([[0, 0, 1],
                  [2, 2, 4],
                  [5, 5, 3]],
                  index=['s1', 's2', 's3'],
                  columns=['o1', 'o2', 'o3'])
    _mapping = pd.DataFrame([['a', 'control'],
                        ['b', 'control'],
                        ['c', 'diseased'],
                        ['d', 'diseased']],
                        index=['s1', 's2', 's3', 's4'],
                        columns=['Barcode', 'Treatment'])
    m2=pd.DataFrame([[2, 4, 4],
                 [5, 3, 3],
                 [0, 0, 1]],
                index=['s2', 's3', 's4'],
                columns=['c1', 'c2', 'c3'])
    assert(match_samples(m1, m2, _mapping)[0].index==['s2','s3'])

In [33]:
mic_match, met_match, mapping_match=match_samples(microb.T, metab.T, mapping)

In [34]:
print(mic_match.shape)
print(met_match.shape)
print(mapping_match.shape)

(182, 735)
(182, 1710)
(182, 69)


In [39]:
met_match = met_match.reset_index().rename(columns = {'index': '#SampleID'})
mic_match = mic_match.reset_index().rename(columns = {'index': '#SampleID'})
mapping_match = mapping_match.reset_index()

In [44]:
# save matched tables for downstream analysis
mapping_match.to_csv("./data/haddad_6weeks_metadata_matched.txt", sep ='\t', index = False)
mic_match.to_csv("./data/haddad_6weeks_deblur_otus_rare2k_matched.txt", sep ='\t', index = False)
met_match.to_csv("./data/haddad_6weeks_allFeatures_pqn_matched.txt", sep ='\t', index = False)

In [5]:
# convert matched feature tables to biom
! biom convert -i ./data/haddad_6weeks_deblur_otus_rare2k_matched.txt \
  -o ./data/haddad_6weeks_deblur_otus_rare2k_matched.biom --table-type "OTU table" \
  --to-hdf5

! biom convert -i ./data/haddad_6weeks_allFeatures_pqn_matched.txt \
  -o ./data/haddad_6weeks_allFeatures_pqn_matched.biom --table-type "OTU table" \
  --to-hdf5